In [ ]:
path = 'C:\Users\Horo\.cache\kagglehub\datasets\bhavikjikadara\dog-and-cat-classification-dataset\versions\1'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import VGG16


# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of the data for validation
)

# Training data generator
train_data = train_datagen.flow_from_directory(
    #'data/cats_and_dogs/training',
    'dataset/training_set',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Validation data generator
val_data = train_datagen.flow_from_directory(
    #'data/cats_and_dogs/validation',
    'dataset/test_set',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Build the CNN model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu',kernel_initializer='he_uniform'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

for layer in base_model.layers:
    layer.trainable = False

opt = SGD(learning_rate=0.001, momentum=0.9)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
# Train the model
history = model.fit(
    train_data,
    epochs=50,
    validation_data=val_data,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_data)
print(f'Validation Accuracy: {val_accuracy}')


# Save the trained model to disk
model.save('cat_dog_classifier_SGD_2.h5')
